In [1]:
import datetime
import pandas as pd
from igf_data.igfdb.igfTables import Base, Seqrun, Seqrun_stats, Platform
from igf_data.igfdb.baseadaptor import BaseAdaptor
from igf_data.igfdb.platformadaptor import PlatformAdaptor
from igf_data.igfdb.seqrunadaptor import SeqrunAdaptor
from sqlalchemy import create_engine

In [2]:
dbparams={'dbhost':'0.0.0.0', \
          'dbuser':'root', \
          'dbpass':'xxxx', \
          'dbname':'igfdb', \
          'driver':'mysql', \
          'connector':'pymysql'}

In [3]:
base=BaseAdaptor(**dbparams)
session_class=base.get_session_class()

In [4]:
Base.metadata.drop_all(base.engine)
Base.metadata.create_all(base.engine)

In [5]:
platform_data=[{'platform_igf_id':'ILM4K_001', \
                'model_name':'HISEQ4000',\
                'vendor_name':'ILLUMINA', \
                'software_name':'RTA', \
                'software_version':'RTA2'},
               {'platform_igf_id':'ILMNS_001', \
                'model_name':'NEXTSEQ',\
                'vendor_name':'ILLUMINA', \
                'software_name':'RTA', \
                'software_version':'RTA2'},
               {'platform_igf_id':'ILMMS_001', \
                'model_name':'MISEQ',\
                'vendor_name':'ILLUMINA', \
                'software_name':'RTA', \
                'software_version':'RTA1.18.64'},
              ]

In [6]:
pl=PlatformAdaptor(**{'session_class':base.session_class})
pl.start_session()
pl.store_platform_data(data=platform_data)

In [7]:
seqrun_data=[{'seqrun_igf_id':'170627_K00345_0012_AHJJKTBBXX', 
              'flowcell_id':'AHJJKTBBXX', 
              'platform_igf_id':'ILM4K_001',
              'lib_type':'type1'},
             {'seqrun_igf_id':'170627_M03291_0071_000000000-D0V30', 
              'flowcell_id':'D0V30', 
              'platform_igf_id':'ILMMS_001',
              'lib_type':'type2' },
            ]

In [8]:
sra=SeqrunAdaptor(**{'session_class':base.session_class})
sra.start_session()
sra.store_seqrun_and_attribute_data(data=seqrun_data)

In [9]:
sra_data=sra.fetch_records(sra.session.query(Seqrun))

In [10]:
sra_data

,seqrun_id,seqrun_igf_id,reject_run,date_created,flowcell_id,platform_id
0,1,170627_K00345_0012_AHJJKTBBXX,N,2017-07-25 18:23:28,AHJJKTBBXX,1
1,2,170627_M03291_0071_000000000-D0V30,N,2017-07-25 18:23:28,D0V30,3


In [11]:
sra.close_session()
pl.close_session()

In [16]:
seqrun_stats_data=[ {'seqrun_id':'1', 
                     'lane_number':'1', 
                     'bases_mask':'y150n1,i8,i8,y150n1',
                     'undetermined_barcodes':{'ATTTA+GGGC':50, 'TTTA+CCCG':150},
                     'known_barcodes':{'AAAAA+TTTT':10, 'CCCC+GGGG':20},
                     'undetermined_fastqc':{'read1':{'number':130},'read2':{'number':130}},
                    }   
                 ]

In [19]:
base.start_session()
base.store_records(table=Seqrun_stats, data=seqrun_stats_data, mode='serial')
base.commit_session()

In [20]:
base.close_session()

In [24]:
base.start_session()
data=base.fetch_records(query=base.session.query(Seqrun_stats))
base.close_session()

In [27]:
data.known_barcodes

0    {'CCCC+GGGG': 20, 'AAAAA+TTTT': 10}
Name: known_barcodes, dtype: object